# Wine Price Predictor<a name = 'beginning'></a>    


In this project I will apply EDA, missing value processing, and categorical data coding. The project is purely a learning project.    


The data was taken from the Kaggle database [kaggle.com](https://www.kaggle.com/)  

### Problem statement  

> to determine the price of wine   
    
### Goals of this project   

* Practice EDA: Data Loading, Data Familiarization, Data Cleaning;   
* Processing Categorical Variables.    

### Steps   

* [Data Loading](#loading)   
* [Getting Familiar With the Data](#familiar)    
* [Missing Values Processing](#missing_values)      
* [Preparing Data for Training](#preparing)    
* [Coding the Categorical Variables](#coding_cv)   
* [Training the Model](#training)  

In [87]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
from pandas.core.common import SettingWithCopyWarning
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

### Data Loading<a name = 'loading'></a>   

[The Beginning](#beginning)  

In [52]:
data_path = 'C:/Users/samur/Desktop/My Machine Learning/Projects/data/winemag-data-130k-v2.csv'

data = pd.read_csv(data_path, index_col = 0)

### Getting Familiar With the Data<a name = 'familiar'></a>   

[The Beginning](#beginning)  

In [53]:
print(data.shape, '\n')

print(data.dtypes)

data.head()

(129971, 13) 

country                   object
description               object
designation               object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


Let's collect the categorical and numerical columns into separate variables for convenience later on.

In [54]:
# Get cols with object type
object_cols = [col for col in data.columns if data[col].dtype == 'object']

# Get cols with numerical type
num_cols = [col for col in data.columns if data[col].dtype != 'object']

# Check that everything works correctly
print("Object cols: ", object_cols, '\n')
print('Numerical cols: ', num_cols)

Object cols:  ['country', 'description', 'designation', 'province', 'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title', 'variety', 'winery'] 

Numerical cols:  ['points', 'price']


### Missing Values Processing<a name = 'missing_values'></a>  

[The Beginning](#beginning)  

I'll use an **Imputation with Most Frequent Value** approach for categorical data   

In [55]:
# Let's check how many empty fields there are in each categorical column
for col in object_cols:
    nan_count = data[col].isnull().sum()
    print(col, ': ', nan_count, ' from ', len(data))

country :  63  from  129971
description :  0  from  129971
designation :  37465  from  129971
province :  63  from  129971
region_1 :  21247  from  129971
region_2 :  79460  from  129971
taster_name :  26244  from  129971
taster_twitter_handle :  31213  from  129971
title :  0  from  129971
variety :  1  from  129971
winery :  0  from  129971


In [56]:
# Create list with the most frequent values for categorical variables

most_frequent_values = []
for col in object_cols:
    most_frequent_values.append(data[col].mode()[0])

    
    
# Put the most frequent values into empty fields

for i in range(len(object_cols)):
    data.loc[data[object_cols[i]].isnull(), object_cols[i]] = most_frequent_values[i]

In [57]:
# Check that there are no empty fields left

for col in object_cols:
    nan_count = data[col].isnull().sum()
    print(col, ': ', nan_count, ' Nan from ', len(data))

country :  0  Nan from  129971
description :  0  Nan from  129971
designation :  0  Nan from  129971
province :  0  Nan from  129971
region_1 :  0  Nan from  129971
region_2 :  0  Nan from  129971
taster_name :  0  Nan from  129971
taster_twitter_handle :  0  Nan from  129971
title :  0  Nan from  129971
variety :  0  Nan from  129971
winery :  0  Nan from  129971


And I will use an **Imputation with Mean** approach for numerical data

In [63]:
# Firstly, let's check how many empty fields there are now
for col in num_cols:
    nan_count = data[col].isnull().sum()
    print(col, ': ', nan_count, ' from ', len(data))


points :  0  from  129971
price :  0  from  129971


In [61]:
data.loc[data['price'].isnull(), 'price'] = data['price'].mean()

In [62]:
# Check that in the data there are no missing values left
print(data.isnull().any())

country                  False
description              False
designation              False
points                   False
price                    False
province                 False
region_1                 False
region_2                 False
taster_name              False
taster_twitter_handle    False
title                    False
variety                  False
winery                   False
dtype: bool


### Preparing Data for Training<a name = 'preparing'></a>   

[The Beginning](#beginning)  

In [88]:
data.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.363389,Sicily & Sardinia,Etna,Central Coast,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.000000,Douro,Napa Valley,Central Coast,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",Reserve,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Central Coast,Alexander Peartree,@vossroger,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [65]:
# Set the target value
y = data.price

# Choose features we want to use in our prediction model. The features were choosen by intuition.
wine_features = ['country', 'points', 'province', 'variety']

# And than we create a specific Frame for the list of the selected features
X = data[wine_features]

The `province` and `variety` columns contain many categories, some of wich are small and therefore unimformative. Let's combine those categories into the `Other` category. 

In [66]:
X.province.value_counts()

California        36310
Washington         8639
Bordeaux           5941
Tuscany            5897
Oregon             5373
                  ...  
Messinia              1
Markopoulo            1
Paardeberg            1
Krania Olympus        1
Neuchâtel             1
Name: province, Length: 425, dtype: int64

In [69]:
# Count the amount of entries of each value in the 'province column
province_counts = X.province.value_counts()

# Find values that appear less than 100 times
province_values_to_replace = province_counts[province_counts < 100].index

# Replace them with 'Other'
X.province = X.province.replace(province_values_to_replace, 'Other')

X.province.nunique()

84

Now we have 84 categories in the `province` column instead of 425. Let's do the same thing for the `variety` column.

In [70]:
X.variety.value_counts()

Pinot Noir                    13273
Chardonnay                    11753
Cabernet Sauvignon             9472
Red Blend                      8946
Bordeaux-style Red Blend       6915
                              ...  
Cabernet Sauvignon-Barbera        1
Sauvignonasse                     1
Forcallà                          1
Meseguera                         1
Bobal-Cabernet Sauvignon          1
Name: variety, Length: 707, dtype: int64

In [71]:
# Count the amount of entries of each value in the variety column
variety_counts = X.variety.value_counts()

# Find values that appear less than 100 times
variety_values_to_replace = variety_counts[variety_counts < 100].index

# Replace them with 'Other'
X.variety = X.variety.replace(variety_values_to_replace, 'Other')

# Print the result
X.variety.nunique()  

93

Now let's split the data into training and validation sets.

In [74]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Coding the Categorical Variables<a name = 'coding_cv'></a>   

[The Beginning](#beginning)  

In [76]:
object_features = [col for col in X_train.columns if X_train[col].dtype == 'object']

print(object_features)

['country', 'province', 'variety']


In [78]:
# Transforming categorical data with LabelEncoder. I've choosen this one, because I have a lot of categories

my_le = LabelEncoder()


# Training of the encoder on the training set
for col in object_features:
    X_train.loc[:, col + '_encoded'] = my_le.fit_transform(X_train[col])
    if set(X_val[col]).issubset(set(X_train[col])):
        # The trained coder is applied, when all the categories in training and validation sets match
        X_val.loc[:, col + '_encoded'] = my_le.transform(X_val[col])
    else:
        # The "Other" category is applied for new categories in the validation dataset
        X_val.loc[:, col + '_encoded'] = my_le.transform(X_val[col].replace(set(X_val[col]) 
                                                                    - set(X_train[col]), 'Other')).copy()
    
    
# In the code above I don't work with slice od DataFrame, I've created a deep copy. So I'll ignore 'SettingWithCopyWarning'
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

Now when we have encoded categorical columns, we can drop original ones

In [80]:
X_train = X_train.drop(columns = ['country', 'variety', 'province'])
X_val = X_val.drop(columns = ['country', 'variety', 'province'])

### Training the Model<a name = 'training'></a>  

[The Beginning](#beginning)  

In [86]:
model = RandomForestRegressor(random_state=42)

model.fit(X_train, y_train)

preds = model.predict(X_val)

mean_absolute_error(y_val, preds)

13.178348545580882